## Import of Libraries

In [2]:
import cv2
import numpy as np
import sys

## Reading the two Images

In [4]:
# reading the two images
img1=cv2.imread("images/set1/img1.png")
img2=cv2.imread("images/set1/img2.png")

# converting the images to grayscale
img1_gray=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2_gray=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

# cv2.imshow("img1_gray",img1_gray)
# cv2.imshow("img2_gray",img2_gray)

# cv2.waitKey(0)


## Detecting distinctive keypoints and descriptors

In [5]:
#initiated ORB for 200 points i.e. when the image is being processed the ORB would look out for 200 keypoints and descriptors
orb=cv2.ORB_create(nfeatures=2000)

#unpacking keypionts and descriptors
key1,des1=orb.detectAndCompute(img1,None)
key2,des2=orb.detectAndCompute(img2,None)
# here key1,key2 are the list of  keypoints and des1,des2 are the list of  descriptors

# draw only keypoints location,not size and orientation
# cv2.imshow("img1",cv2.drawKeypoints(img1, key1, None, (255, 0, 255)))
# cv2.imshow("img2",cv2.drawKeypoints(img2, key2, None, (255, 0, 255)))

           
# cv2.waitKey(0)

## Matching the Keypoints between two Images

In [6]:
# Create a BFMatcher object.
# It will find all of the matching keypoints on two images
bf = cv2.BFMatcher_create(cv2.NORM_HAMMING)

# Find matching points
matches = bf.knnMatch(des1, des2,k=2)

## Match overlapping keypoints with draw_matches()

In [7]:
def draw_matches(img1, key1, img2, key2, matches):
    r, c = img1.shape[:2]
    r1, c1 = img2.shape[:2]
    
    
    # Create a blank image with the size of the first image + second image
    output_img = np.zeros((max([r, r1]), c+c1, 3), dtype='uint8')
    output_img[:r, :c, :] = np.dstack([img1, img1, img1])
    output_img[:r1, c:c+c1, :] = np.dstack([img2, img2, img2])
    
    # Go over all of the matching points and extract them
    for match in matches:
        img1_idx = match.queryIdx
        img2_idx = match.trainIdx
        (x1, y1) = key1[img1_idx].pt
        (x2, y2) = key2[img2_idx].pt
        
        # Draw circles on the keypoints
        cv2.circle(output_img, (int(x1),int(y1)), 4, (0, 255, 255), 1)
        cv2.circle(output_img, (int(x2)+c,int(y2)), 4, (0, 255, 255), 1)
        
        # Connect the same keypoints
        cv2.line(output_img, (int(x1),int(y1)), (int(x2)+c,int(y2)), (0, 255, 255), 1)
        
    return output_img
        
        
    
  

In [8]:
all_matches = []
for m, n in matches:
    all_matches.append(m)
    
img3 = draw_matches(img1_gray, key1, img2_gray, key2, all_matches[:30])

cv2.imshow("match",img3)
cv2.waitKey(0)

-1

In [9]:
# Finding the best matches
good = []
for m, n in matches:
    if m.distance < 0.6 * n.distance:
        good.append(m)
        
# cv2.imshow("good points img1",cv2.drawKeypoints(img1, [key1[m.queryIdx] for m in good], None, (255, 0, 255)))
cv2.imshow("good points img2",cv2.drawKeypoints(img2, [key2[m.trainIdx] for m in good], None, (255, 0, 255)))

cv2.waitKey(0)

-1

## Stitching the two images together

In [10]:
def warpImages(img1, img2, H):
    rows1, cols1 = img1.shape[:2]
    rows2, cols2 = img2.shape[:2]
    
    list_of_points_1 = np.float32([[0,0], [0, rows1],[cols1, rows1], [cols1, 0]]).reshape(-1, 1, 2) 
    # coordinates of the reference image (img1)
    temp_points = np.float32([[0,0], [0,rows2], [cols2,rows2], [cols2,0]]).reshape(-1,1,2)
    # coordinates of the second image(img2) that we want to transform
    
    # When we have established a homography we need to warp perspective
    # Change field of view
    list_of_points_2 = cv2.perspectiveTransform(temp_points, H)
    
    list_of_points = np.concatenate((list_of_points_1,list_of_points_2), axis=0)
    
    # unpacking the boundary x, y co-ordinates of the resultant stitched image
    [x_min, y_min] = np.int32(list_of_points.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(list_of_points.max(axis=0).ravel() + 0.5)
    
    translation_dist = [-x_min,-y_min]
    
    H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])
    
    output_img = cv2.warpPerspective(img2, H_translation.dot(H), (x_max-x_min, y_max-y_min))
    
    
    output_img[translation_dist[1]:rows1+translation_dist[1], translation_dist[0]:cols1+translation_dist[0]] = img1
    
    return output_img

In [11]:
MIN_MATCH_COUNT = 10

if len(good) > MIN_MATCH_COUNT:
    # Convert keypoints to an argument for findHomography
    src_pts = np.float32([ key1[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    dst_pts = np.float32([ key2[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    
    # Establish a homography
    M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    
    result = warpImages(img2, img1, M)
    
    
    cv2.imshow("img1",img1)
    cv2.imshow("img2",img2)
    cv2.imshow("result",result)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()